In [ ]:
# Install required Python libraries
try:
    import subprocess
    subprocess.check_call(["pip", "install", "torch", "onnx", "numpy", "hummingbird-ml"])
except:
    print("Ensure all dependencies are installed.")

from sklearn.linear_model import LinearRegression
from hummingbird.ml import convert
import numpy as np
import json

In [ ]:
# Create sample data
X = np.array([[1, 2], [2, 3], [3, 4], [4, 5]])  # Input features
y = np.dot(X, np.array([1, 2])) + 3  # Target output: y = 1*x1 + 2*x2 + 3

# Train a simple linear regression model
reg_model = LinearRegression().fit(X, y)
print(f"Model score: {reg_model.score(X, y)}")

# Convert the trained model to ONNX format
onnx_model = convert(reg_model, "torch", X[:1]).model
onnx_path = "linear_regression.onnx"

# Export ONNX model
import torch
dummy_input = torch.randn(1, X.shape[1])
torch.onnx.export(
    onnx_model, dummy_input, onnx_path,
    input_names=["input"], output_names=["output"],
    opset_version=13
)
print(f"ONNX model exported to {onnx_path}")


In [ ]:
# Prepare input data for proof generation
data_path = "input.json"
sample_input = X[:1]  # Use one sample from the dataset
input_data = sample_input.flatten().tolist()  # Flatten for JSON serialization

# Save the input data to a JSON file
data = [input_data]  # Wrap in outer array [[]]
with open(data_path, "w") as f:
    json.dump(data, f, indent=4)
print(f"Input data saved to {data_path}")


In [ ]:
import subprocess

# Paths for CLI commands
proof_path = "proof.json"

# Command for proof generation
# Make sure to generate the binary first: `cargo build --release`
cmd = [
    "../../target/release/mina-zkml-cli", "proof",
    "-m", onnx_path,
    "-i", data_path,
    "-o", proof_path,
    "--input-visibility", "public",
    "--output-visibility", "public"
]

# Run the CLI command
result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(result.stdout)
if result.returncode == 0:
    print(f"Proof successfully generated at {proof_path}")
else:
    print(f"Error generating proof: {result.stderr}")


In [ ]:
# Extract the "output" field from proof.json
output_path = "output.json"
try:
    with open(proof_path, "r") as proof_file:
        proof_data = json.load(proof_file)
    if "output" in proof_data:
        output_data = proof_data["output"]
        with open(output_path, "w") as output_file:
            json.dump(output_data, output_file, indent=4)
        print(f"Output data successfully saved to {output_path}")
    else:
        print("No 'output' field found in proof.json")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
# Create a public output file from the proof
try:
    # Load proof.json
    with open(proof_path, "r") as proof_file:
        proof_data = json.load(proof_file)

    # Extract the "output" field
    if "output" in proof_data:
        output_data = proof_data["output"]
        
        # Save the output data to output.json
        with open(output_path, "w") as output_file:
            json.dump(output_data, output_file, indent=4)
        
        print(f"Output data successfully saved to {output_path}")
    else:
        print("No 'output' field found in proof.json")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Command for proof verification
cmd = [
    "../../target/release/mina-zkml-cli", "verify",
    "-m", onnx_path,
    "-i", data_path,
    "-p", proof_path,
    "-o", output_path,
    "--input-visibility", "public",
    "--output-visibility", "public"
]

# Run the CLI command
result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print(result.stdout)
if result.returncode == 0:
    print(f"Proof successfully verified at {proof_path}")
else:
    print(f"Error verifying proof: {result.stderr}")
